#Building an image retrieval system with deep features


#Fire up GraphLab Create

In [1]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [2]:
image_train = graphlab.SFrame('image_train_data/')

[INFO] This non-commercial license of GraphLab Create is assigned to sdhar.anlyt@gmail.comand will expire on October 05, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-8688 - Server binary: C:\Users\sumit.dhar\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\SUMIT~1.DHA\AppData\Local\Temp\graphlab_server_1445913703.log.0
[INFO] GraphLab Server Version: 1.6.1


In [4]:
image_test = graphlab.SFrame('image_test_data/')

#Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [3]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [5]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


In [6]:
image_train['deep_features'][0]

array('d', [0.24287176132202148, 1.0954537391662598, 0.0, 0.39362990856170654, 0.0, 0.0, 11.894915580749512, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5788496136665344, 0.4954667389392853, 2.5141289234161377, 0.0, 1.5180106163024902, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5813961029052734, 0.0, 0.0, 2.595609426498413, 2.7079553604125977, 0.0, 0.0, 0.0, 0.8509902954101562, 0.0, 0.7203489542007446, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2700355052947998, 0.0, 0.0, 0.0, 0.0, 0.08592796325683594, 0.0, 0.7010231018066406, 0.0, 0.0, 0.0, 0.0, 0.024805665016174316, 0.0, 0.0, 0.17549043893814087, 0.0, 0.0, 0.0, 0.0, 0.0, 2.392784595489502, 0.0, 0.0, 4.471865653991699, 0.0, 1.6358323097229004, 0.0, 4.417484760284424, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4117904901504517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1247677206993103, 0.0, 0.0, 0.8957164287567139, 0.0, 0.0, 0.3334987759590149, 0.0, 0.0, 0.20787304639816284, 2.5435447692871094, 

#Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [7]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


#Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [8]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [9]:
knn_model.query(cat)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 5.001ms      |
PROGRESS: | Done         |         | 100         | 159.01ms     |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [10]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [11]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 7ms          |
PROGRESS: | Done         |         | 100         | 177.01ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [12]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

##Finding similar images to a car

In [14]:
car = image_train[8:9]
car['image'].show()

In [15]:
get_images_from_ids(knn_model.query(car))['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 9.001ms      |
PROGRESS: | Done         |         | 100         | 160.009ms    |
PROGRESS: +--------------+---------+-------------+--------------+


#Just for fun, let's create a lambda to find and show nearest neighbor images

In [16]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [17]:
show_neighbors(8)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 6.001ms      |
PROGRESS: | Done         |         | 100         | 150.003ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [18]:
show_neighbors(31)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 10ms         |
PROGRESS: | Done         |         | 100         | 152.005ms    |
PROGRESS: +--------------+---------+-------------+--------------+


#Assignment

In [20]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [21]:
image_train['label'].unique()

dtype: str
Rows: 4
['bird', 'dog', 'cat', 'automobile']

In [22]:
bird_train = image_train[image_train['label'] == 'bird']

In [23]:
dog_train = image_train[image_train['label'] == 'dog']

In [24]:
cat_train = image_train[image_train['label'] == 'cat']

In [25]:
auto_train = image_train[image_train['label'] == 'automobile']

In [26]:
bird_model = graphlab.nearest_neighbors.create(bird_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [27]:
cat_model = graphlab.nearest_neighbors.create(cat_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [28]:
dog_model = graphlab.nearest_neighbors.create(dog_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [29]:
auto_model = graphlab.nearest_neighbors.create(auto_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [30]:
image_test = graphlab.SFrame('image_test_data/')

In [34]:
get_images_from_ids(dog_model.query(cat1))['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 14ms         |
PROGRESS: | Done         |         | 100         | 83.004ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [36]:
image_test[0:1]['image'].show()

In [32]:
cat1 = image_test[0:1]

In [33]:
cat_model.query(cat1)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 9ms          |
PROGRESS: | Done         |         | 100         | 67.004ms     |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [37]:
get_images_from_ids(dog_model.query(cat1))['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 10.001ms     |
PROGRESS: | Done         |         | 100         | 74.005ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [39]:
get_images_from_ids(cat_model.query(cat1))['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 12ms         |
PROGRESS: | Done         |         | 100         | 63.003ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [32]:
dog_model.query(cat1)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 8.001ms      |
PROGRESS: | Done         |         | 100         | 69.004ms     |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [39]:
cat_model.query(cat1)['distance'].mean()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 8ms          |
PROGRESS: | Done         |         | 100         | 68.004ms     |
PROGRESS: +--------------+---------+-------------+--------------+


36.15573070978294

In [40]:
dog_model.query(cat1)['distance'].mean()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 8.001ms      |
PROGRESS: | Done         |         | 100         | 75.004ms     |
PROGRESS: +--------------+---------+-------------+--------------+


37.77071136184156

In [41]:
lbl = image_train['label'].unique()

In [43]:
lbl

dtype: str
Rows: 4
['bird', 'dog', 'cat', 'automobile']

In [55]:
lbl[1]+'_test'
len(lbl)

4

In [75]:
for i in range(len(lbl)):
    a = lbl[i]
    graphlab.sFrame(a+'_test'= image_test[image_test['label'] == 'a'])

SyntaxError: keyword can't be an expression (<ipython-input-75-05ada33857c8>, line 3)

In [40]:
bird_test = image_train[image_train['label'] == 'bird']
cat_test = image_train[image_train['label'] == 'cat']
dog_test = image_train[image_train['label'] == 'dog']
auto_test = image_train[image_train['label'] == 'automobile']

In [78]:
dog_cat = cat_model.query(dog_test, k=1)
dog_dog = dog_model.query(dog_test, k=1)
dog_bird = bird_model.query(dog_test, k=1)
dog_auto = auto_model.query(dog_test, k=1)
cat_cat = cat_model.query(cat_test, k=1)
cat_dog = dog_model.query(cat_test, k=1)
cat_bird = bird_model.query(cat_test, k=1)
cat_auto = auto_model.query(cat_test, k=1)
cat_test = cat_model.query(test_data, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 509          | 64643   | 24.9509     | 201.007ms    |
PROGRESS: | Done         | 259081  | 100         | 252.009ms    |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 509          | 64643   | 24.9509     | 201.006ms    |
PROGRESS: 

NameError: name 'test_data' is not defined

In [51]:
dog_auto_neighbors.head()

NameError: name 'dog_auto_neighbors' is not defined

In [45]:
dog_distances = graphlab.SFrame({'dog_auto': dog_auto['distance'],
                              'dog_bird': dog_bird['distance'],
                              'dog_cat': dog_cat['distance'],
                              'dog_dog': dog_dog['distance']
                             })
cat_distances = graphlab.SFrame({'cat_auto': cat_auto['distance'],
                              'cat_bird': cat_bird['distance'],
                              'cat_cat': cat_cat['distance'],
                              'cat_dog': cat_dog['distance']
                             })

In [53]:
def is_dog_correct(row):
  if min(dog_distances[row].values())== dog_distances[row]['dog_dog']:
        return (1)
  else:
        return (0)

In [59]:
def is_cat_correct(row):
  if min(cat_distances[row].values())== cat_distances[row]['cat_cat']:
        return (1)
  else:
        return (0)

In [52]:
dog_distances.apply(lambda row: 1 if row['dog_dog'] == min(row.values()) else 0).sum()

509L

In [72]:
cat_distances.apply(lambda row: if row['cat_cat'] == min(row.values()) 1 else 0).sum()

SyntaxError: invalid syntax (<ipython-input-72-f542bcc7a50c>, line 1)

In [76]:
j = 0
for i in range(cat_distances.num_rows()):
    j = j + is_cat_correct(i)


In [77]:
print j

509
